In [7]:
from Bio import SeqIO

In [8]:
from Bio.Data import CodonTable

In [9]:
import pandas as pd

## 1. Document Dr. X's function with comments and with markdown text in your Jupyter notebook

Dr. X's function is to extract sequences from files, it takes a file and returns sequences that paired with species names in a dictionary

In [4]:
def get_sequences_from_file(fasta_fn): # define a function to get sequences from files
    sequence_data_dict = {} # create an empty dictionary
    for record in SeqIO.parse(fasta_fn, "fasta"): # takes a filename and format name, and returns a SeqRecord iterator
        description = record.description.split() # split the description in the SeqRecord object, save to a new variable description
        species_name = description[1] + " " + description[2] # extract the second and third element from the description, save to a new variable species_name 
        sequence_data_dict[species_name] = record.seq # assign the sequence in the SeqRecord object in the dictionary using species_name as the key
    return(sequence_data_dict) # return the dictionary with species name and sequences

## 2. Write a function that translates a string of nucleotides to amino acids 

* pseudo-code and suggestions
def translate_function(string_nucleotides): 
mito_table = CodonTable.unambiguous_dna_by_name["Vertebrate Mitochondrial"] # this should work using BioPython (be sure to check what this returns)
for-loop through every 3rd position in string_nucleotides to get the codon using range subsets
IMPORTANT: if the sequence has a stop codon at the end, you should leave it off
this is how you can retrieve the amino acid: mito_table.forward_table[codon]
add the aa to aa_seq_string
return(aa_seq_string)

In [11]:
def translate_function(string_nucleotides): 
    mito_table = CodonTable.unambiguous_dna_by_name["Vertebrate Mitochondrial"]
    aa_seq_string = ''
    for i in range(0, len(string_nucleotides), 3):
        codon = string_nucleotides[i:i+3]
        aa_seq_string += mito_table[codon]
    return(aa_seq_string)

In [12]:
translate_function("ATGCTA")

TypeError: 'NCBICodonTableDNA' object is not subscriptable